In [5]:
import os
import tensorflow as tf
import tf2onnx
import onnxruntime as ort
from tensorflow.keras.models import load_model

# 3. Set up paths
model_dir = r"D:\FCDS\semster 8\IOT 2\Projects\project 2\tinyml-weather-classification\models"
onnx_dir = os.path.join(model_dir, "onnx")

# Create ONNX directory if it doesn't exist
os.makedirs(onnx_dir, exist_ok=True)

onnx_model_path = os.path.join(onnx_dir, "weather_model.onnx")

# 4. Load the trained Keras model
keras_model_path = os.path.join(model_dir, "tensorflow", "weather_model.h5")
model = load_model(keras_model_path)

# 5. Convert using tf2onnx - METHOD 1 (Recommended)
# Define input signature (shape must match your model input)
input_signature = [tf.TensorSpec(shape=[None, 2], dtype=tf.float32, name='input')]

# Convert model
onnx_model, _ = tf2onnx.convert.from_keras(
    model, 
    input_signature=input_signature,
    opset=13
)

# Save the ONNX model
with open(onnx_model_path, 'wb') as f:
    f.write(onnx_model.SerializeToString())

print(f"ONNX model successfully saved to:\n{onnx_model_path}")

# 6. Verify the file exists
if not os.path.exists(onnx_model_path):
    raise FileNotFoundError(f"ONNX model was not created at {onnx_model_path}")

# 7. Verify the ONNX model
try:
    ort_session = ort.InferenceSession(onnx_model_path)
    print("\nONNX model verification successful!")
    print("Input details:", ort_session.get_inputs())
    print("Output details:", ort_session.get_outputs())
    
    # Test with sample data
    sample_input = {ort_session.get_inputs()[0].name: [[15.0, 0.65]]}  # Example: 15°C, 65% humidity
    prediction = ort_session.run(None, sample_input)
    print("\nSample prediction:", prediction)
    
except Exception as e:
    print(f"\nError verifying ONNX model: {e}")
    print("Possible solutions:")
    print("1. Check if the input shape matches your model")
    print("2. Try different opset version (e.g., 11-15)")
    print("3. Verify TensorFlow and tf2onnx versions are compatible")

ONNX model successfully saved to:
D:\FCDS\semster 8\IOT 2\Projects\project 2\tinyml-weather-classification\models\onnx\weather_model.onnx

ONNX model verification successful!
Input details: [<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x000001E0CEBE9DF0>]
Output details: [<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x000001E0CEB19E70>]

Sample prediction: [array([[1.]], dtype=float32)]
